In [1]:
import types, yaml, re
from pathlib import Path
from typing import Any, Dict, List, Optional
from mlx_lm.utils import load, save_config
from datasets import load_dataset
#from mlx_lm.utils import load, save_config

#model, tokenizer = load('mlx-community/Llama-3.2-1B-Instruct-bf16')

/Users/irf/Desktop/SynologyDrive/git/agent/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
main_path = Path.cwd()
model_list_path = Path(main_path /'config' /'config.yaml')
with open(model_list_path, 'r') as file:
    model_list = yaml.load(file, yaml.SafeLoader)
model_list['models']


['Llama-3.2-1B-Instruct-bf16']

In [4]:
dataset_name = "lcw99/wikipedia-korean-20240501-1million-qna"
system_context = 'context'
user_context = 'question'
assistant_context = 'answer'
model_name = model_list['models'][0]

current_path = Path.cwd()
dataset_path = Path(current_path /'data'/ dataset_name.split('/')[-1])
dataset_path.mkdir(parents=True, exist_ok=True)


In [5]:
model_list

{'models': ['Llama-3.2-1B-Instruct-bf16']}

In [6]:
model_list.get ('datasets',[])

[]

In [7]:
ds = model_list.get ('datasets',[])
if dataset_name not in ds:
    ds.append(dataset_name)
    model_list['datasets'] = ds
    with open(model_list_path, 'w') as file:
        yaml.dump(model_list, file)

In [8]:
ds1 =load_dataset(dataset_name, split='train[:80%]')
ds2 =load_dataset(dataset_name, split='train[80%:-10%]')
ds3 =load_dataset(dataset_name, split='train[-10%:]')
# Define a function to apply the chat template
def apply_chat_template(example):
    messages = [
        {"role": "system", "content": example[system_context]},
        {"role": "user", "content": example[user_context]},
        {"role": "assistant", "content": example[assistant_context]}
    ]
    return {"messages":messages}

# Apply the chat template function to the dataset
nds1 = ds1.map(apply_chat_template)
nds2 = ds2.map(apply_chat_template)
nds3 = ds3.map(apply_chat_template)


nds1.to_json(str(dataset_path) + '/train.jsonl')
nds2.to_json(str(dataset_path) + '/test.jsonl')
nds3.to_json(str(dataset_path) + '/validation.jsonl')


Creating json from Arrow format: 100%|██████████| 100/100 [00:04<00:00, 23.20ba/s]


960264298